<a href="https://colab.research.google.com/github/littlecanargie/EpidemicModeling/blob/main/20210126_Practice_1_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machinery (takes some time)

## Base packages

In [ ]:
libraries = c("dplyr", "magrittr", "tidyr", "ggplot2", "readxl")  
for(x in libraries) { library(x, character.only=TRUE, warn.conflicts=FALSE, quietly=TRUE) }

install.packages("zoo"); library("zoo")

R.Version()$version.string

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Data hub for COVID19 datasets
## install the package
install.packages("COVID19")
## load the package
library("COVID19")

## Stan

In [ ]:
# Step 1: install CmdStanR and only the packages that it directly depends on.
install.packages('versions')
library(versions)
# Install package CmdStanR from GitHub
library(devtools)
if(!require(cmdstanr)){
  devtools::install_github("stan-dev/cmdstanr", dependencies=c("Depends", "Imports"))
  library(cmdstanr)
}

In [ ]:
# Step 2: download and untar the CmdStan binary for Google Colab instances
## Install CmdStan binaries
if (!file.exists("cmdstan-2.23.0.tgz")) {
  system("wget https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz", intern=T)
  system("tar zxf colab-cmdstan-2.23.0.tar.gz", intern=T)
}
list.files("cmdstan-2.23.0")

In [ ]:
# Step 3: Register the CmdStan install location
## Set cmdstan_path to CmdStan installation
set_cmdstan_path("cmdstan-2.23.0")
## helper function
print_file <- function(file, nlines=-1L) {
  cat(paste(readLines(file, n=nlines), "\n", sep=""), sep="")
}

In [ ]:
# will be used for postprocessing
if (!require("devtools")) {
  install.packages("devtools")
}
devtools::install_github("stan-dev/bayesplot")

library(posterior)
library(bayesplot)

## Templates for Stan which will be used later

In [ ]:
# Stan program
stan_code = "functions {
    /* calculating the convolutions */
    // X: first function, Yrev: reversed version of the second function
    // K: length of X and Yrev
    // the result is the vector of length K-1, because the first element equal zero is omitted
    vector convolution(vector X, vector Yrev, int K) {
        vector[K-1] res;
        res[1] = X[1]*Yrev[K];
        for (k in 2:K-1) 
            res[k] = dot_product(head(X, k), tail(Yrev, k)); // by definition of the convolution

        return res;        
    }
}

data {

}

parameters {
}

model {
}"
stan_file = write_stan_file(stan_code)

# data file
data_list = list(param1_SI = 2.305, param2_SI = 5.452)
data_file = tempfile(fileext = ".json")
write_stan_json(data_list, data_file)

# # Compile example model
# mod = cmdstan_model(stan_file)

# # Condition on data
# fit <- mod$sample(data = data_file, iter_sampling=1000, iter_warmup=500, chains=3) #, seed=123

# # Print a summary of the posterior sample
# options(digits = 2)
# fit$summary()

# output = fit$summary(c("Rt"), ~quantile(.x, probs = c(0.5, 0.025, 0.975))) %>% as.data.frame
# names(output) = c("variable", "median", "lower", "upper")
# output %>% head

# out = stringr::str_match_all(output$variable, "[\\d+(.*)]")
# output$index = sapply(out, function(x) paste(x[,1],collapse=""))
# output %<>% mutate(variable = stringr::str_extract(variable, "^[^\\[]+")) %>% select(variable, index, everything())
# output %>% head